# Mini-Compiler Project Phase 2
Ali Osman	                211001561

Mohamed Ashraf Qushta       211001221

Saif Eldeen Sameh           211001048

In [7]:
import json

# Load tokens generated from lexical analysis
with open('token_output.json', 'r') as token_file:
    tokens = [
        {"token": line.split(", ")[0].split(": ")[1].strip().lower(), "lexeme": line.split(", ")[1].split(": ")[1].strip()} 
        for line in json.load(token_file)
    ]

    print(tokens)

[{'token': 'let', 'lexeme': 'LET'}, {'token': 'identifier', 'lexeme': 'a'}, {'token': 'equal', 'lexeme': '='}, {'token': 'number', 'lexeme': '5'}, {'token': 'let', 'lexeme': 'LET'}, {'token': 'identifier', 'lexeme': 'b'}, {'token': 'equal', 'lexeme': '='}, {'token': 'left_brack', 'lexeme': '['}, {'token': 'number', 'lexeme': '1'}, {'token': 'comma', 'lexeme': ','}, {'token': 'number', 'lexeme': '2'}, {'token': 'comma', 'lexeme': ','}, {'token': 'number', 'lexeme': '3'}, {'token': 'right_brack', 'lexeme': ']'}, {'token': 'let', 'lexeme': 'LET'}, {'token': 'identifier', 'lexeme': 'c'}, {'token': 'equal', 'lexeme': '='}, {'token': 'identifier', 'lexeme': 'a'}, {'token': 'operator', 'lexeme': '+'}, {'token': 'number', 'lexeme': '10'}, {'token': 'call', 'lexeme': 'CALL'}, {'token': 'identifier', 'lexeme': 'myFunction'}, {'token': 'left_paren', 'lexeme': '('}, {'token': 'identifier', 'lexeme': 'a'}, {'token': 'comma', 'lexeme': ','}, {'token': 'identifier', 'lexeme': 'b'}, {'token': 'right_p

In [8]:
class ParseNode:
    def __init__(self, type, value=None):
        self.type = type
        self.value = value
        self.children = []

    def add_child(self, node):
        if node:
            self.children.append(node)

    def to_string(self, level=0):
        indent = "|   " * level
        result = f"{indent}{self.type}"
        if self.value:
            result += f": {self.value}"
        result += "\n"

        for child in self.children:
            result += child.to_string(level + 1)

        return result

In [9]:
# Define the Grammar Rules
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_token_index = 0
        self.errors = []  # For syntax errors

    def current_token(self):
        if self.current_token_index < len(self.tokens):
            return self.tokens[self.current_token_index]
        return None

    def match(self, expected_type):
        token = self.current_token()
        if token is None:
            self.errors.append(f"Unexpected end of input. Expected {expected_type} .at index {self.current_token_index}")
            return None
        if token['token'] == expected_type:
            self.current_token_index += 1
            return ParseNode(expected_type, token['lexeme'])
        else:
            self.errors.append(f"Expected {expected_type}, found {token['token']} at token: {token} .at index {self.current_token_index}")
            return None

    def parse_program(self):
        program_node = ParseNode("Program")

        statements_block = self.statements_block()
        if statements_block:
            program_node.add_child(statements_block)

        return program_node

    def statements_block(self):
        block_node = ParseNode("StatementsBlock")

        while self.current_token_index < len(self.tokens):
            token = self.current_token()
            if token is None : #or token['token'] in ['else', 'endif', 'endwhile', 'endfor', 'until', 'end']
                break
            
            statement = self.parse_statements()
            if statement:
                block_node.add_child(statement)

        return block_node

    def parse_statements(self):
        token = self.current_token()
        if token is None:
            self.errors.append("Unexpected end of input while parsing a statement .at index {self.current_token_index}")
            return None
        if token['token'] == 'let':
            return self.parse_let_statement()
        elif token['token'] == 'call':
            return self.parse_call_statement()
        elif token['token'] == 'while':
            return self.parse_while_loop()
        elif token['token'] == 'do':
            return self.parse_do_while_loop()
        elif token['token'] == 'for':
            return self.parse_for_loop()
        elif token['token'] == 'repeat':
            return self.parse_repeat_until_loop()
        elif token['token'] == 'func':
            return self.parse_function_definition()
        elif token['token'] == 'if':
            return self.parse_if_statement()
        elif token['token'] == 'identifier':
            return self.parse_identifier_statement()
        else:
            self.errors.append(f"Unexpected token: {token['token']} .at index {self.current_token_index}")
            self.current_token_index += 1
            return None

    def parse_statement(self):
        children = []
        while self.current_token() and self.current_token()['token'] in (
            'let', 'call', 'while', 'do', 'for', 'repeat', 'func', 'if', 'identifier'
        ):
            statement = self.parse_statements()
            if statement:
                children.append(statement)
        return children


        
    def parse_let_statement(self):
        node = ParseNode("LetStatement")
        node.add_child(self.match('let'))
        node.add_child(self.match('identifier'))
        node.add_child(self.match('equal'))

        if self.current_token() and self.current_token()['token'] == 'left_brack':
            node.add_child(self.parse_list_elements())
        else:
            expression = self.parse_expression()
            if expression:
                if len(expression.children) == 1:
                    for child in expression.children:
                        node.add_child(child)
                else:
                    node.add_child(expression)

        return node

    def parse_call_statement(self):
        node = ParseNode("CallStatement")
        node.add_child(self.match('call'))
        node.add_child(self.match('identifier'))
        node.add_child(self.match('left_paren'))

        args_node = ParseNode("Arguments")
        while self.current_token() and self.current_token()['token'] != 'right_paren':
            args_node.add_child(self.parse_expression())
            if self.current_token() and self.current_token()['token'] == 'comma':
                args_node.add_child(self.match('comma'))

        node.add_child(args_node)
        node.add_child(self.match('right_paren'))

        return node

    def parse_expression(self):
        token = self.current_token()
        if token is None:
            self.errors.append("Unexpected end of input while parsing an expression .at index {self.current_token_index}")
            return None
        if token['token'] in ['identifier', 'number', 'string', 'listelement']:
            single_node = self.match(token['token'])
            if self.current_token() and self.current_token()['token'] in ['operator', 'compound_operator']:
                # Create an Expression node only if there's more than one token
                node = ParseNode("Expression")
                node.add_child(single_node)
                node.add_child(self.match(self.current_token()['token']))
                node.add_child(self.parse_expression())
                return node
            else:
                return single_node
        elif token['token'] == 'left_brack':
            return self.parse_list_elements()
        else:
            self.errors.append(f"Unexpected token in expression: {token['token']} .at index {self.current_token_index}")
            return None

    def parse_list_elements(self):
        node = ParseNode("ListElements")
        node.add_child(self.match('left_brack'))

        while self.current_token() and self.current_token()['token'] != 'right_brack':
            element = self.parse_expression()
            if element:
                node.add_child(element)

            if self.current_token() and self.current_token()['token'] == 'comma':
                node.add_child(self.match('comma'))

        node.add_child(self.match('right_brack'))
        return node

    def parse_while_loop(self):
        """Parses a WHILE loop."""
        node = ParseNode("WhileLoop")
        node.add_child(self.match('while'))

        # Parse the condition
        condition = self.parse_expression()
        if condition:
            node.add_child(condition)

        node.add_child(self.match('do'))

        # Parse multiple statements in the body
        while self.current_token() and self.current_token()['token'] != 'endwhile':
            statement = self.parse_statements()
            if statement:
                if isinstance(statement, list):
                    for stmt in statement:
                        node.add_child(stmt)
                else:
                    node.add_child(statement)

        node.add_child(self.match('endwhile'))
        return node

    def parse_do_while_loop(self):
        """Parses a DO-WHILE loop."""
        node = ParseNode("DoWhileLoop")
        node.add_child(self.match('do'))

        # Parse multiple statements in the body
        while self.current_token() and self.current_token()['token'] != 'while':
            statement = self.parse_statements()
            if statement:
                if isinstance(statement, list):
                    for stmt in statement:
                        node.add_child(stmt)
                else:
                    node.add_child(statement)

        node.add_child(self.match('while'))

        # Parse the condition
        condition = self.parse_expression()
        if condition:
            node.add_child(condition)

        return node


    def parse_for_loop(self):
        node = ParseNode("ForLoop")
        node.add_child(self.match('for'))
        node.add_child(self.match('identifier'))

        if self.current_token() and self.current_token()['token'] == 'equal':
            node.add_child(self.parse_classic_for_loop())
        elif self.current_token() and self.current_token()['token'] == 'in':
            node.add_child(self.parse_range_based_for_loop())
        else:
            self.errors.append(f"Expected 'equal' or 'in', found {self.current_token()['token']} .at index {self.current_token_index}")

        node.add_child(self.match('do'))

        body = self.parse_statement()
        if body:
            for stmt in body:
                node.add_child(stmt)

        node.add_child(self.match('endfor'))
        return node

    def parse_classic_for_loop(self):
        node = ParseNode("ClassicForLoop")
        node.add_child(self.match('equal'))
        node.add_child(self.parse_expression())
        node.add_child(self.match('to'))
        node.add_child(self.parse_expression())

        if self.current_token() and self.current_token()['token'] == 'step':
            node.add_child(self.match('step'))
            node.add_child(self.parse_expression())

        return node

    def parse_range_based_for_loop(self):
        node = ParseNode("RangeBasedForLoop")
        node.add_child(self.match('in'))
        node.add_child(self.match('range'))
        node.add_child(self.match('left_paren'))
        node.add_child(self.parse_expression())
        node.add_child(self.match('comma'))
        node.add_child(self.parse_expression())
        node.add_child(self.match('comma'))
        node.add_child(self.parse_expression())
        node.add_child(self.match('right_paren'))
        return node

    def parse_repeat_until_loop(self):
        node = ParseNode("RepeatUntilLoop")
        node.add_child(self.match('repeat'))

        body = self.parse_statement()
        if body:
            for stmt in body:
                node.add_child(stmt)

        node.add_child(self.match('until'))

        condition = self.parse_expression()
        if condition:
            node.add_child(condition)

        return node

    def parse_function_definition(self):
        node = ParseNode("Function")
        node.add_child(self.match('func'))
        node.add_child(self.match('identifier'))
        node.add_child(self.match('left_paren'))

        param_list = ParseNode("ParameterList")
        while self.current_token() and self.current_token()['token'] != 'right_paren':
            param_list.add_child(self.match('identifier'))
            if self.current_token() and self.current_token()['token'] == 'comma':
                param_list.add_child(self.match('comma'))

        node.add_child(param_list)
        node.add_child(self.match('right_paren'))
        node.add_child(self.match('begin'))

        body = self.parse_statement()
        if body:
            for stmt in body:
                node.add_child(stmt)

        node.add_child(self.match('return'))
        node.add_child(self.parse_expression())
        node.add_child(self.match('end'))

        return node

    def parse_if_statement(self):
        node = ParseNode("IfStatement")
        node.add_child(self.match('if'))

        condition = self.parse_expression()
        if condition:
            node.add_child(condition)

        node.add_child(self.match('then'))

        true_block = self.parse_statement()
        if true_block:
            for stmt in true_block:
                node.add_child(stmt)

        if self.current_token() and self.current_token()['token'] == 'else':
            node.add_child(self.match('else'))
            false_block = self.parse_statement()
            if false_block:
                for stmt in false_block:
                    node.add_child(stmt)

        node.add_child(self.match('endif'))
        return node

    def parse_identifier_statement(self):
        token = self.current_token()
        if token is None:
            self.errors.append("Unexpected end of input while parsing an identifier statement .at index {self.current_token_index}")
            return None
        if token['token'] == 'identifier':
            if self.current_token_index + 1 < len(self.tokens):
                next_token = self.tokens[self.current_token_index + 1]
                if next_token['token'] == 'compound_operator':
                    return self.parse_compound_assignment()
                elif next_token['token'] == 'increment':
                    return self.parse_increment()
                elif next_token['token'] == 'decrement':
                    return self.parse_decrement()
        self.errors.append(f"Unexpected identifier usage at token: {token} .at index {self.current_token_index}")
        self.current_token_index += 1
        return None

    def parse_compound_assignment(self):
        node = ParseNode("CompoundAssignment")
        node.add_child(self.match('identifier'))
        node.add_child(self.match('compound_operator'))
        node.add_child(self.parse_expression())
        return node

    def parse_increment(self):
        node = ParseNode("Increment")
        node.add_child(self.match('identifier'))
        node.add_child(self.match('increment'))
        return node

    def parse_decrement(self):
        node = ParseNode("Decrement")
        node.add_child(self.match('identifier'))
        node.add_child(self.match('decrement'))
        return node

# Run the parser
parser = Parser(tokens)
parse_tree = parser.parse_program()

if parser.errors:
    print("Syntax Errors: "+parser.errors[0])

else:
    print("Parse Tree:")
    print(parse_tree.to_string())


Parse Tree:
Program
|   StatementsBlock
|   |   LetStatement
|   |   |   let: LET
|   |   |   identifier: a
|   |   |   equal: =
|   |   |   number: 5
|   |   LetStatement
|   |   |   let: LET
|   |   |   identifier: b
|   |   |   equal: =
|   |   |   ListElements
|   |   |   |   left_brack: [
|   |   |   |   number: 1
|   |   |   |   comma: ,
|   |   |   |   number: 2
|   |   |   |   comma: ,
|   |   |   |   number: 3
|   |   |   |   right_brack: ]
|   |   LetStatement
|   |   |   let: LET
|   |   |   identifier: c
|   |   |   equal: =
|   |   |   Expression
|   |   |   |   identifier: a
|   |   |   |   operator: +
|   |   |   |   number: 10
|   |   CallStatement
|   |   |   call: CALL
|   |   |   identifier: myFunction
|   |   |   left_paren: (
|   |   |   Arguments
|   |   |   |   identifier: a
|   |   |   |   comma: ,
|   |   |   |   identifier: b
|   |   |   right_paren: )
|   |   IfStatement
|   |   |   if: IF
|   |   |   Expression
|   |   |   |   identifier: a
|   |   |   |   o

<h3>
<center>
    Context Free Grammar
</center>
</h3>
<h5>
Program &rarr; Statements_block <br>

Statements_block &rarr; 
    Let_statement       |
    Call_statement      |
    If_statment         |
    Expression          |
    Condition           |
    While_loop          |
    Do_while            |
    For_loop            |
    Repeat_until        |
    Function            |
    List_operations     |
    Compound_Assignment |
    Increment           |
    Decrement<br>

Statments &rarr; Statements_block Statments\`<br>
Statments` &rarr; Statments | 
                  &epsilon;<br> 

Let_statement &rarr; let identifier = Let_statement\`<br>
Let_statement` &rarr; Expression      |
            [ Element_List ]<br>

List_operations &rarr; identifier [ Expression ] <br>

Element_List &rarr; Expression Element_List\`<br>
Element_List` &rarr; , Element_List | 
               &epsilon;<br>

Expression &rarr; Variable Expression\`<br>
Expression` &rarr; Operations Expression   |
                   &epsilon;<br> 

Call_statement &rarr; call identifier ( ArgList )  <br>

ArgList &rarr; Expression ArgList\`<br>
ArgList` &rarr; , ArgList | 
               &epsilon;<br>  

Variable &rarr; identifier  |
                number      |
                string<br>

Condition &rarr; Variable Condition\`<br>
Condition` &rarr; Operations Condition |
                  &epsilon;<br>

While_loop &rarr; while Condition do Statments endwhile<br>

Do_while &rarr; do Statments while Condition<br>

For_loop &rarr; for identifier For_Loop\` do Statments endfor<br>
For_Loop` &rarr; Classic_For_Loop   |
                 Range_For_Loop<br>

Classic_For_Loop &rarr; = Expression to Expression Expression Increment<br>
Range_For_Loop &rarr; in range( Expression , Expression , Expression )<br>

Repeat_until &rarr; repeat Statments until Condition <br>

If_statment &rarr; if Condition then Statments else Statments endif <br>

Function &rarr; func identifier ( ParamList ) begin Statments return Expression end<br>

ParaList &rarr; identifier ParaList\`<br>
ParaList` &rarr; , ParaList |
               &epsilon;<br>

Compound_Assignment &rarr; identifier Compound_Operator Expression <br>
Compound_Operator &rarr; += | -= | *= | /= <br>

Increment &rarr; identifier ++ <br>

Decrement &rarr; identifier -- <br>
</h5>